In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

In [2]:
import pyspark
sc = pyspark.SparkContext()

In [3]:
print(sc)

In [17]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 10)

[429, 115, 521, 537, 646, 126, 219, 666, 800, 12]

In [5]:
%pwd

'/home/jovyan/work'

In [6]:
%ls

Dockerfile  Untitled.ipynb


In [7]:
%cd /data

/data


In [8]:
%ls

allen-p     donohoe-t       jones-t          neal-s         skilling-j
arnold-j    dorland-c       kaminski-v       nemec-g        smith-m
arora-h     ermis-f         kean-s           panus-s        south-s
badeer-r    farmer-d        keavey-p         pereira-s      stclair-c
bailey-s    fischer-m       king-j           perlingiere-d  stepenovitch-j
bass-e      fossum-d        kuykendall-t     pimenov-v      storey-g
baughman-d  gay-r           lavorato-j       presto-k       sturm-f
beck-s      germany-c       lay-k            quenet-j       symes-k
benson-r    gilbertsmith-d  lenhart-m        ring-a         taylor-m
brawner-s   giron-d         lewis-a          rodrique-r     tholt-j
buy-r       griffith-j      linder-e         rogers-b       townsend-j
campbell-l  grigsby-m       lokay-m          ruscitti-k     tycholiz-b
carson-m    guzman-m        love-p           sager-e        ward-k
cash-m      haedicke-m      maggi-m          saibi-e        weldon-c
corman-s    hain-m          

In [9]:
import nltk

In [20]:
import json

data_dir = '/data'

for person in os.listdir(data_dir):
    with open(os.path.join(data_dir, person), encoding='utf-8') as f:
        dt = json.load(f)
    # todo: stop data in dataframe.
    # Ik weet nog niet of de sqlContext de meest handige context is
    # https://spark.apache.org/docs/1.0.1/sql-programming-guide.html
    p = sqlContext.createDataFrame(dt)
    p.registerAsTable(person.replace('-', ''))

/usr/local/spark/python/pyspark/sql/context.py:237: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"
/usr/local/spark/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


In [21]:
cashm = sqlContext.sql("SELECT * FROM cashm")
dates = cashm.map(lambda d: 'date: {}'.format(d))
for date in dates.collect():
    print(date)

date: Row(date='2000-05-12 02:21:00-07:00', doc_id='cash-m/237.', source='cash-m', text="FYI.  Here is most of the information.  Michelle\n\n----- Forwarded by Michelle Cash/HOU/ECT on 05/12/2000 09:21 AM -----\n\n\tBryce Baxter\n\t05/12/2000 07:49 AM\n\t\t \n\t\t To: Michelle Cash/HOU/ECT@ECT\n\t\t cc: Susan Harrison/HOU/ECT@ECT, Brenda F Herod/HOU/ECT@ECT, Lisa \nCsikos/HOU/ECT@ECT, Rita Wynne/HOU/ECT@ECT\n\t\t Subject: New Vendors for Contract Accounting Services\n\n\n\t\n\n\nMichelle, as I mentioned in my voice mail to you, I would like to use \ncontractors from the two firms listed below.  Both of these firms have \ncontractors available now, so as soon as we get these agreements in place we \ncan bring them in and put them to work.  If possible, I'd like any agreement \nwe put in place to be flexible enough to allow us to use other contractors in \nthe future besides those we currently have identified.  If you need me to do \nanything else or if you are not the right person to ta